<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/survey_agent_asks_questions_according_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Survey Agent Asks Questions According Form

## Demo Description

**Author:** Agently Team

**Prompt Language:** English

**Agent Components:** Role

**Description:**

This case shows how to use Agently framework to create a survey agent asking questions according the survey form, communicate with customer, collecting feedbacks using natural language. During the survey, customer may tell a story or jump out of the question to say something else and the agent can try its best to handle this complex situation. This case shows how agent handle complex task and maybe you can modified the demo code to make it work better?

本案例演示了如何使用Agently框架创建一个问卷调查agent，通过自然语言问答的方式和客户交流，回收问卷信息。在交流过程中，客户有可能会跳出当前的问题讲一些别的事情，这对于agent而言会是一个挑战。可以看看agent能否处理这些复杂的场景。这个案例展现了agent处理复杂任务的一种思路，你可以在这个基础上进一步优化，或许你的思路会让agent做得更好？

## Step 1: Install Packages

In [ ]:
!pip install Agently

## Step 2: Demo Code

In [45]:
from datetime import datetime
import Agently

agent_factory = Agently.AgentFactory()

agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "" })\
    .set_settings("model.OpenAI.options", { "model": "gpt-3.5-turbo-1106" })

survey = {
    "name": {
        "value":  None,
        "customer_explation": [],
        "data_type": "String",
        "optional": False,
    },
    "phone": {
        "value": None,
        "customer_explation": [],
        "data_type": "Number",
        "optional": True,
    },
    "dining time": {
        "value":  None,
        "customer_explation": [],
        "data_type": "Date String",
        "optional": True,
    },
    "satisfaction": {
        "value": None,
        "customer_explation": [],
        "data_type": "Number: 1-10, 10 is the best, 1 is the worst",
        "optional": True,
    },
    "favourite food": {
        "value": None,
        "customer_explation": [],
        "data_type": "String",
        "optional": True,
    },
    "commendation": {
        "value": None,
        "customer_explation": [],
        "data_type": "String",
        "optional": True,
    },
    "criticism": {
        "value": None,
        "customer explation": [],
        "data_type": "String",
        "optional": True,
    },
    "comments_and_add_up": {
        "value": None,
        "customer_explation": [],
        "data_type": "Boolean",
        "optional": True,
        "comments": "It's for add up information during the survey. Make sure update the full value each time you need to update",
    },
}

def start_survey(survey: dict):
    agent = agent_factory.create_agent()
    # Prepare Basic Information for Agent
    agent\
        .set_role("Role", "Green City Kitchen Survey Expert")\
        .set_role("NAME", "Jack Grayson")\
        .general(
            "Introduce of Green City Kitchen",
            "1457, Sunshine Street, Fake City." + \
            "Beautiful view of the seaside." + \
            "Open from 10:30am to 10:30pm " + \
            "providing brunch, lunch and dinner." + \
            "Ordering Number: 35671881" + \
            "Party pre-order accepted."
        )\
        .general(
            "Current Date",
            datetime.now().date()
        )
    # Start the Survey
    can_stop = False
    chat_history = []
    opening = agent\
        .input({
            "survey": survey,
        })\
        .output({
            "opening": (
                "String",
                "Your warm opening about this survey " + \
                "and introduce the background of this survey " + \
                "like who you are and which dining customer had is being reviewed"
            ),
        })\
        .start()
    print("[Survey Agent]: ", opening["opening"])
    chat_history.append({ "role": "assistant", "content": opening["opening"] })
    while not can_stop:
        # Planning: Check Survey Form and Ask Question
        question_plan = agent\
            .chat_history(chat_history)\
            .instruct(
                "Action Rules",
                [
                    "1. Check {survey} form and make sure all filled items have complete and correct information.",
                    "2. If there're incomplete or incorrect information, choose one of the items and asking customer again.",
                    "3. If there're still blank items, choose one of the blank items to ask.",
                    "4. Check chat history, make sure ask even optional items at least once",
                    "5. Do remember to check chat history and do not repeat questions that are already asked and responsed correctly.",
                ]
            )\
            .input({
                "survey": survey,
            })\
            .output({
                "incorrect_items": ("Array | Null", "list items names with incorrect information."),
                "target_item": ("String", "choose one target item for this question round. If there're incorrect items, ask them first."),
                "question": ("String", "your question about {target_item} to customer. Check chat history, don't repeat yourself."),
            })\
            .start()
        # Asking: Survey Agent Interact with Customer
        print("[Survey Agent]: ", question_plan["question"])
        need_to_dig_deep = True
        while need_to_dig_deep:
            customer_response = input(f"[Customer]:  ")
            # Filling the Form: Survey Agent Analyses the Response, Fills the Form and Replies.
            analysis = agent\
                .chat_history(chat_history)\
                .input({
                    "survey": survey,
                    "question_target_item": question_plan["target_item"],
                    "question": question_plan["question"],
                    "customer_response": customer_response,
                })\
                .output({
                    "fillings": [{
                        "target_item": ("String", "according {customer_response}, which item from {survey} is response about?"),
                        "value": ("As required", "value you will fill into the form about {target_item} according {survey} required"),
                        "customer_explation": ("String | Null", "Extra explation from the customer"),
                    }],
                    "need_to_dig_deep": ("Boolean", "if {customer_response} is not so exactly and according chat history you think you can get more information, return true"),
                    "reply": "Your reply to customer, to finish this item's question-answer or question to dig deep",
                })\
                .start()
            need_to_dig_deep = analysis["need_to_dig_deep"]
            if "fillings" in analysis:
                for filling in analysis["fillings"]:
                    survey[filling["target_item"]]["value"] = filling["value"]
                    if "customer_explation" in filling\
                        and filling["customer_explation"] not in (None, ""):
                        survey[filling["target_item"]]["customer_explation"]\
                            .append(str(filling["customer_explation"]))
            print("[Survey Agent]: ", analysis["reply"])
            chat_history.extend([
                { "role": "assistant", "content": question_plan["question"] },
                { "role": "user", "content": customer_response },
                { "role": "assistant", "content": analysis["reply"] }
            ])
        # Can Stop Check
        can_stop_judgement = agent\
            .input({
                "survey": survey
            })\
            .output({
                "can_stop_judgement": (
                    "Boolean",
                    "if all survey required items are filled " + \
                    "and optional items are tried to ask in chat history, " + \
                    "you can stop."
                ),
                "stop_reply": ("String", "if {hang_up_judgement} is true, generate your thanks and goodbye reply"),
            })\
            .start()
        if can_stop_judgement["can_stop_judgement"]:
            can_stop = True
            print("[Survey Agent]: ", can_stop_judgement["stop_reply"])
            chat_history.append({ "role": "assistant", "content": can_stop_judgement["stop_reply"]})
            print("[Survey Form]:\n", survey)

start_survey(survey)

[Survey Agent]:  Hello! Thank you for taking the time to participate in our survey. We are conducting this survey to gather feedback from our valued customers on their dining experience at our restaurant. Your input will help us improve our services and better cater to your needs. We appreciate your feedback and look forward to hearing about your dining experience with us.
[Survey Agent]:  Thank you for participating in our survey! To begin, could you please provide your name?
[Customer]:  Sam Green
[Survey Agent]:  Thank you, Sam, for providing your name. Now I'd like to ask if you'd be willing to share your phone number with us for follow-up purposes?
[Survey Agent]:  Thank you, Sam. Would you be comfortable providing us with your phone number for follow-up purposes? This will help us ensure that we can address any issues or feedback you may have. If you prefer not to share your phone number, that's completely understandable and your feedback in this survey is still incredibly valuab

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)